In [2]:
import geopandas as gpd
import pandas as pd
import geopandas
import shapely.geometry as geom
from polygon_geohasher.polygon_geohasher import polygon_to_geohashes, geohashes_to_polygon
import pygeohash as gh
%matplotlib inline
import geohash
import geopandas as gp
import pandas as pd
import math
import geojson
from geojson import MultiLineString
from shapely.geometry import MultiLineString
from shapely.geometry import MultiPoint
import numpy as np
from shapely import geometry
from shapely.geometry import Point, Polygon, box,LineString
from geopandas import datasets, GeoDataFrame, read_file
from geopandas.tools import overlay
from matplotlib import pyplot
import matplotlib.pyplot as plt
from shapely.geometry.polygon import LinearRing, Polygon
from numpy import cos,sin,arccos
import numpy as np
from pylab import *
from shapely.ops import cascaded_union
from shapely.geometry import Point, LineString, mapping
import scipy.interpolate
import geopy
import geopy.distance
import time
import networkx as nx
from sklearn.model_selection import train_test_split
import datetime as dt

In [3]:
weekend_busy_G = nx.read_gpickle("weekend_busy_G.gpickle")
weekend_normal_G = nx.read_gpickle("weekend_normal_G.gpickle")
weekday_busy_G = nx.read_gpickle("weekday_busy_G.gpickle")
weekday_normal_G = nx.read_gpickle("weekday_normal_G.gpickle")
main_graph_G = nx.read_gpickle("main_graph_G.gpickle")


may_weather = pd.read_csv('MAY_SG_hourly_weather.csv')
sg_geofabrik = pd.read_csv('sg_geofabrik.csv')
may_weather['date'] = pd.to_datetime(may_weather['date'], format='%Y%m%d')
may_weather['day'] = may_weather['date'].dt.day
may_weather['time'] = may_weather['time'].astype(str)
may_weather[['hour','minute']] = may_weather['time'].str.split(':',expand=True)
may_weather['hour'] = may_weather['hour'].astype(int)

NameError: name 'apr_weather' is not defined

In [21]:
def test_endpoint(latitude_origin, longitude_origin, latitude_destination, longitude_destination, timestamp, hour_of_day, day_of_week):
    geohash_source = gh.encode(latitude_origin,longitude_origin, precision=6)
    geohash_dest = gh.encode(latitude_destination,longitude_destination, precision=6)
    temp_arr = []
    rain_weight = 0
    poi_weight = 0
    maxspeed_weight = 0
    rain_bool = may_weather.loc[(may_weather['day'] == day_of_week )& (may_weather['hour'] == hour_of_day)]['rain']
    if rain_bool is True:
        rain_weight = 0.15
    try:
        if hour_of_day in busy_hours and day_of_week in weekend_num:
            path = nx.dijkstra_path(weekend_busy_G,source=source_geohash, target=dest_geohash, weight='weight')
            total_weight = 0
            for i in range(len(path) - 1):
                poi_num = sg_geofabrik.loc[sg_geofabrik['geohash'] == path[i]]['no_poi']
                max_speed = sg_geofabrik.loc[sg_geofabrik['geohash'] == path[i]]['maxspeed']
                if len(poi_num) > 0 and poi_num > 5:
                    poi_weight = 0.15
                if len(max_speed) > 0 and max_speed > 50:
                    maxspeed_weight = 0.15
                total_weight += weekend_busy_G[path[i]][path[i+1]]['weight'] * (1 + rain_weight + poi_weight + maxspeed_weight)
            

        elif hour_of_day not in busy_hours and day_of_week in weekend_num:
            path = nx.dijkstra_path(weekend_normal_G,source=geohash_source, target=geohash_dest, weight='weight')
            total_weight = 0
            for i in range(len(path) - 1):
                poi_num = sg_geofabrik.loc[sg_geofabrik['geohash'] == path[i]]['no_poi']
                max_speed = sg_geofabrik.loc[sg_geofabrik['geohash'] == path[i]]['maxspeed']
                if len(poi_num) > 0 and poi_num > 5:
                    poi_weight = 0.15
                if len(max_speed) > 0 and max_speed > 50:
                    maxspeed_weight = 0.15
                total_weight += weekend_normal_G[path[i]][path[i+1]]['weight'] * (1 + rain_weight + poi_weight + maxspeed_weight)
            
        

        elif hour_of_day in busy_hours and day_of_week not in weekend_num:
            path = nx.dijkstra_path(weekday_busy_G,source=geohash_source, target=geohash_dest, weight='weight')
            total_weight = 0
            for i in range(len(path) - 1):
                poi_num = sg_geofabrik.loc[sg_geofabrik['geohash'] == path[i]]['no_poi']
                max_speed = sg_geofabrik.loc[sg_geofabrik['geohash'] == path[i]]['maxspeed']
                if len(poi_num) > 0 and poi_num > 5:
                    poi_weight = 0.15
                if len(max_speed) > 0 and max_speed > 50:
                    maxspeed_weight = 0.15
                total_weight += weekday_busy_G[path[i]][path[i+1]]['weight'] * (1 + rain_weight + poi_weight + maxspeed_weight)
       
        elif hour_of_day not in busy_hours and day_of_week not in weekend_num:
            path = nx.dijkstra_path(weekday_normal_G,source=geohash_source, target=geohash_dest, weight='weight')
            total_weight = 0
            for i in range(len(path) - 1):
                poi_num = sg_geofabrik.loc[sg_geofabrik['geohash'] == path[i]]['no_poi']
                max_speed = sg_geofabrik.loc[sg_geofabrik['geohash'] == path[i]]['maxspeed']
                if len(poi_num) > 0 and poi_num > 5:
                    poi_weight = 0.15
                if len(max_speed) > 0 and max_speed > 50:
                    maxspeed_weight = 0.15
                total_weight += weekday_normal_G[path[i]][path[i+1]]['weight'] * (1 + rain_weight + poi_weight + maxspeed_weight)
            
    except:

        try:
            path = nx.dijkstra_path(main_graph_G,source=geohash_source, target=geohash_dest, weight='weight')
            total_weight = 0
            for i in range(len(path) - 1):
                poi_num = sg_geofabrik.loc[sg_geofabrik['geohash'] == path[i]]['no_poi']
                max_speed = sg_geofabrik.loc[sg_geofabrik['geohash'] == path[i]]['maxspeed']
                if len(poi_num) > 0 and poi_num > 5:
                    poi_weight = 0.15
                if len(max_speed) > 0 and max_speed > 50:
                    maxspeed_weight = 0.15
                total_weight += main_graph_G[path[i]][path[i+1]]['weight'] * (1 + rain_weight + poi_weight + maxspeed_weight)
            
        except:
            print('Error for' + str(test_case[1]['trj_id']))
            pass
            #temp_arr.append(test_case[1]['trj_id'])
            #temp_arr.append(test_case[1]['time_taken'])
            #temp_arr.append(1246.4185714285713)
            #accuracy_arr.append(temp_arr)
    
    return total_weight

In [20]:
print(test_endpoint(1.301775, 103.799257, 1.358001, 103.845160, '1554943236', 0, 3))

887.0


## Example of internal testing done.

In [5]:
test = pd.read_csv('processed_csv/test_set.csv')
test.head()

,trj_id,source_lat,source_lng,source_timestamp,dest_lat,dest_lng,dest_timestamp,time_taken,geohash_source,geohash_dest,hour,day
0,10,1.301775,103.799257,2019-04-09 11:25:47,1.358001,103.845160,2019-04-09 11:44:56,1149.0,w21z6hb,w21zep1,11,1
1,10001,1.375079,103.833412,2019-04-17 11:56:15,1.328267,103.750637,2019-04-17 12:17:05,1250.0,w21zff3,w21z8cu,11,2
2,10010,1.325336,103.726036,2019-04-21 04:49:21,1.408185,103.895839,2019-04-21 05:28:30,2349.0,w21z836,w23bh0q,4,6
3,10018,1.336971,103.706409,2019-04-17 03:12:03,1.288159,103.794293,2019-04-17 03:37:10,1507.0,w21xxgk,w21z3fs,3,2
4,10087,1.401270,103.880918,2019-04-08 09:55:14,1.288438,103.793752,2019-04-08 10:12:48,1054.0,w21zgz5,w21z3fs,9,0


In [4]:
busy_hours = [7, 8, 9, 17, 18, 19]
weekend_num = [5, 6]
accuracy_arr = []
for test_case in test.iterrows():
    temp_arr = []
    rain_weight = 0
    poi_weight = 0
    maxspeed_weight = 0
    #rain_bool = may_weather.loc[(may_weather['day'] == test_case[1]['day'] )& (may_weather['hour'] == test_case[1]['hour'])]['rain']
    try:
        if test_case[1]['hour'] in busy_hours and test_case[1]['day'] in weekend_num:
            path = nx.dijkstra_path(weekend_busy_G,source=test_case[1]['geohash_source'], target=test_case[1]['geohash_dest'], weight='weight')
            total_weight = 0
            for i in range(len(path) - 1):
                total_weight += weekend_busy_G[path[i]][path[i+1]]['weight']
            temp_arr.append(test_case[1]['trj_id'])
            temp_arr.append(test_case[1]['time_taken'])
            temp_arr.append(total_weight)
            accuracy_arr.append(temp_arr)

        elif test_case[1]['hour'] not in busy_hours and test_case[1]['day'] in weekend_num:
            path = nx.dijkstra_path(weekend_normal_G,source=test_case[1]['geohash_source'], target=test_case[1]['geohash_dest'], weight='weight')
            total_weight = 0
            for i in range(len(path) - 1):
                total_weight += weekend_normal_G[path[i]][path[i+1]]['weight']
            temp_arr.append(test_case[1]['trj_id'])
            temp_arr.append(test_case[1]['time_taken'])
            temp_arr.append(total_weight)
            accuracy_arr.append(temp_arr)

        elif test_case[1]['hour'] in busy_hours and test_case[1]['day'] not in weekend_num:
            path = nx.dijkstra_path(weekday_busy_G,source=test_case[1]['geohash_source'], target=test_case[1]['geohash_dest'], weight='weight')
            total_weight = 0
            for i in range(len(path) - 1):
                total_weight += weekday_busy_G[path[i]][path[i+1]]['weight']
            temp_arr.append(test_case[1]['trj_id'])
            temp_arr.append(test_case[1]['time_taken'])
            temp_arr.append(total_weight)
            accuracy_arr.append(temp_arr)

        elif test_case[1]['hour'] not in busy_hours and test_case[1]['day'] not in weekend_num:
            path = nx.dijkstra_path(weekday_normal_G,source=test_case[1]['geohash_source'], target=test_case[1]['geohash_dest'], weight='weight')
            total_weight = 0
            for i in range(len(path) - 1):

                total_weight += weekday_normal_G[path[i]][path[i+1]]['weight']
            temp_arr.append(test_case[1]['trj_id'])
            temp_arr.append(test_case[1]['time_taken'])
            temp_arr.append(total_weight)
            accuracy_arr.append(temp_arr)
    except:

        try:
            path = nx.dijkstra_path(main_graph_G,source=test_case[1]['geohash_source'], target=test_case[1]['geohash_dest'], weight='weight')
            total_weight = 0
            for i in range(len(path) - 1):
                total_weight += main_graph_G[path[i]][path[i+1]]['weight']
            temp_arr.append(test_case[1]['trj_id'])
            temp_arr.append(test_case[1]['time_taken'])
            temp_arr.append(total_weight)
            accuracy_arr.append(temp_arr)
        except:
            print('Error for' + str(test_case[1]['trj_id']))
            pass
            #temp_arr.append(test_case[1]['trj_id'])
            #temp_arr.append(test_case[1]['time_taken'])
            #temp_arr.append(1246.4185714285713)
            #accuracy_arr.append(temp_arr)

AttributeError: 'PytestTester' object has no attribute 'iterrows'